<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#IMDB-Movie-Review-Sentiment-Classification" data-toc-modified-id="IMDB-Movie-Review-Sentiment-Classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>IMDB Movie Review Sentiment Classification</a></span></li><li><span><a href="#Purpose" data-toc-modified-id="Purpose-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Purpose</a></span></li><li><span><a href="#Process" data-toc-modified-id="Process-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Process</a></span></li><li><span><a href="#Configure-notebook,-import-libraries,-and-import-dataset" data-toc-modified-id="Configure-notebook,-import-libraries,-and-import-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Configure notebook, import libraries, and import dataset</a></span></li><li><span><a href="#Examine-the-data" data-toc-modified-id="Examine-the-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Examine the data</a></span></li><li><span><a href="#Cleaning-and-preprocessing" data-toc-modified-id="Cleaning-and-preprocessing-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Cleaning and preprocessing</a></span></li><li><span><a href="#Baseline-Model-development" data-toc-modified-id="Baseline-Model-development-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Baseline Model development</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

<h1>IMDB Movie Review Sentiment Classification</h1>

<img style="float: left; margin-right: 15px; width: 30%; height: 30%;" src="images/imdb.jpg" />

# Purpose

The overall goal of this set of write-ups is to explore a number of machine learning algorithms utilizing natural language processing (NLP) to classify the sentiment in a set of IMDB movie reviews.

The specific goals of this write-up include:
1. Create a set of word embeddings from the IMDb movie review text utilizing [Word2vec](https://en.wikipedia.org/wiki/Word2vec)
2. Cluster the embeddings utilizing a K-nearest neighbors algorithm into a set of centroids
2. Run the models from the [last write-up](./Model-06.ipynb) against the centroid feature set
3. Determine if the centroid feature set improves our ability to correctly classify movie review sentiment

This series of write-ups is inspired by the Kaggle [
Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial) competition.    

Dataset source:  [IMDB Movie Reviews](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)

# Process

Previously covered [here](./Model-06.ipynb#Process).

# Configure notebook, import libraries, and import dataset

##### Import libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from pandas import set_option

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# http://www.nltk.org/index.html
# pip install nltk
import nltk
from nltk.corpus import stopwords

# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# pip install BeautifulSoup4
from bs4 import BeautifulSoup

# https://pypi.org/project/gensim/
# pip install gensim
from gensim.models import word2vec

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

##### Define global variables

In [2]:
seed = 10
np.random.seed(seed)

# Opens a GUI that allows us to download the NLTK data
# nltk.download()

dataPath = os.path.join('.', 'datasets', 'imdb_movie_reviews')
labeledTrainData = os.path.join(dataPath, 'labeledTrainData.tsv')

##### Import dataset

In [3]:
df = pd.read_csv(labeledTrainData, sep = '\t', header = 0, quoting = 3)

# Examine the data

Previously covered [here](./Model-06.ipynb#Examine-the-data).

# Cleaning and preprocessing

Process justification and methodology previously covered [here](./Model-06.ipynb#Cleaning-and-preprocessing).

Define a 'cleaning' function, and clean the training set:

In [4]:
# Convert the stop words to a set
stopWords = set(stopwords.words("english"))

# Clean IMDB review text
def cleanReview(review, stopWords):
    # Remove HTML
    clean = BeautifulSoup(review)
    
    # Remove non-alpha chars
    clean = re.sub("[^a-zA-Z]", ' ', clean.get_text())
    
    # Convert to lower case and "tokenize"
    clean = clean.lower().split()
    
    # Remove stop words
    clean = [x for x in clean if not x in stopWords]

    # Prepare final, cleaned review
    clean = " ".join(clean)
    
    # Return results
    return clean
    

In [5]:
cleanReviews = [cleanReview(x, stopWords) for x in df['review']]
assert(len(df) == (len(cleanReviews)))

# Baseline Model development

We are finally ready to develop the baseline model on the data we've explored, cleaned, and processed.  Because the IMDB data set doesn't include a validation set we'll create one from a portion of the training data.  The processes is similar to our work in previous write-ups such as the [Iris classifier](.//Model-01.ipynb).

In [6]:
# Pull in the labeled data
labeledTrainData = os.path.join(dataPath, 'labeledTrainData.tsv')
df = pd.read_csv(labeledTrainData, sep = '\t', header = 0, quoting = 3)

# Pull in the unlabeled data since it can be utilized by Word2Vec
unlabeledTrainData = os.path.join(dataPath, 'unlabeledTrainData.tsv')
dfUn = pd.read_csv(unlabeledTrainData, sep = '\t', header = 0, quoting = 3)

In [7]:
# Validation
print('df.shape :', df.shape)
print('dfUn.shape :', dfUn.shape)

df.shape : (25000, 3)
dfUn.shape : (50000, 2)


Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

In [8]:
# Update stop word helper function to output a list of words

# Clean IMDB review text
def cleanReview(review, removeStopWords = False):
    # Convert the stop words to a set
    stopWords = set(stopwords.words("english"))
    
    # Remove HTML
    clean = BeautifulSoup(review)
    
    # Remove non-alpha chars
    clean = re.sub("[^a-zA-Z]", ' ', clean.get_text())
    
    # Convert to lower case and "tokenize"
    clean = clean.lower().split()
    
    # Remove stop words
    if removeStopWords:
        clean = [x for x in clean if not x in stopWords]
    
    # Return results
    return clean

In [9]:
# Examine
cleanReview(df.iloc[25,2])[:12]

['looking',
 'for',
 'quo',
 'vadis',
 'at',
 'my',
 'local',
 'video',
 'store',
 'i',
 'found',
 'this']

In [10]:
# Examine
cleanReview(dfUn.iloc[0,1])[:12]

['watching',
 'time',
 'chasers',
 'it',
 'obvious',
 'that',
 'it',
 'was',
 'made',
 'by',
 'a',
 'bunch']

Create function to break review into a list of sentences which are list of words (i.e. list of lists)

In [11]:
# Creating function implementing punkt tokenizer for sentence splitting
import nltk.data

# Only need this the first time...
# nltk.download('punkt')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def createSentences(review, tokenizer, remove_stopwords = False):
    # Init container to hold results
    sentences = []
    
    # Split review string into sentences
    tokenSentences = tokenizer.tokenize(review.strip())

    # Clean the sentences via cleanReview() function
    for s in tokenSentences:
        # If a sentence is empty, skip it
        if len(s) > 0:
            # Clean sentence
            sentences.append( cleanReview( s, remove_stopwords ))
    
    # Return list of clean sentences
    return sentences

In [12]:
# Examine
_ = createSentences(df.iloc[25,2], tokenizer)
print(_[0][:12])
print(len(_))

['looking', 'for', 'quo', 'vadis', 'at', 'my', 'local', 'video', 'store', 'i', 'found', 'this']
8


In [13]:
# Examine
_ = createSentences(dfUn.iloc[0,1], tokenizer)
print(_[0][:12])
print(len(_))

['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch']
5


Now combine the labeled and unlabeled list of lists:

In [14]:
combined = []

for s in df.iloc[:,2]:
    combined += createSentences(s, tokenizer)

In [15]:
for s in dfUn.iloc[:,1]:
    combined += createSentences(s, tokenizer)

Quick examination:

In [16]:
print('len(combined): ', len(combined))
print("\nSample sentence:")
print(combined[0])

len(combined):  795538

Sample sentence:
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


Train the Word2Vec model:

In [17]:
# Set Word2Vec params
features = 300       # Word vector dimensionality                      
minWordCount = 40    # Minimum word count                        
workers = 4          # Number of threads to run in parallel
context = 10         # Context window size                                                                                    
downSampling = 1e-3  # Downsample setting for frequent words

model = word2vec.Word2Vec(combined, 
                          workers=workers,
                          size=features, 
                          min_count = minWordCount,
                          window = context, 
                          sample = downSampling)

# https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.init_sims.html
# If replace is set, forget the original vectors and only keep the normalized ones = saves lots of memory!
# Note that you cannot continue training after doing a replace. 
# The model becomes effectively read-only = you can call most_similar, similarity etc., but not train.
model.init_sims(replace = True)

# Save model to disk
model.save("300features_40minwords_10context")

2018-11-01 14:34:28,272 : INFO : collecting all words and their counts
2018-11-01 14:34:28,272 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-11-01 14:34:28,315 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-11-01 14:34:28,357 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-11-01 14:34:28,397 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-11-01 14:34:28,441 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-11-01 14:34:28,482 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-11-01 14:34:28,524 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-11-01 14:34:28,566 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-11-01 14:34:28,609 : INFO : PROGRESS: 

2018-11-01 14:34:31,540 : INFO : PROGRESS: at sentence #720000, processed 16105489 words, keeping 118221 word types
2018-11-01 14:34:31,585 : INFO : PROGRESS: at sentence #730000, processed 16331870 words, keeping 118954 word types
2018-11-01 14:34:31,629 : INFO : PROGRESS: at sentence #740000, processed 16552903 words, keeping 119668 word types
2018-11-01 14:34:31,673 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2018-11-01 14:34:31,716 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2018-11-01 14:34:31,760 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-11-01 14:34:31,804 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-11-01 14:34:31,850 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-11-01 14:34:31,874 : INFO : collected 123504 word types from a corp

2018-11-01 14:35:19,159 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-01 14:35:19,168 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-01 14:35:19,169 : INFO : EPOCH - 3 : training on 17798082 raw words (12750654 effective words) took 18.4s, 692815 effective words/s
2018-11-01 14:35:20,180 : INFO : EPOCH 4 - PROGRESS: at 7.42% examples, 940288 words/s, in_qsize 7, out_qsize 0
2018-11-01 14:35:21,181 : INFO : EPOCH 4 - PROGRESS: at 13.25% examples, 838239 words/s, in_qsize 5, out_qsize 0
2018-11-01 14:35:22,204 : INFO : EPOCH 4 - PROGRESS: at 20.84% examples, 871527 words/s, in_qsize 8, out_qsize 2
2018-11-01 14:35:23,217 : INFO : EPOCH 4 - PROGRESS: at 27.58% examples, 865464 words/s, in_qsize 7, out_qsize 0
2018-11-01 14:35:24,237 : INFO : EPOCH 4 - PROGRESS: at 34.70% examples, 868652 words/s, in_qsize 8, out_qsize 2
2018-11-01 14:35:25,255 : INFO : EPOCH 4 - PROGRESS: at 41.65% examples, 869165 words/s, in_qsize 7, out_qsize 5
2

Explore the results:

In [18]:
model.most_similar("great")

[('terrific', 0.730087399482727),
 ('fantastic', 0.7199629545211792),
 ('wonderful', 0.7167843580245972),
 ('superb', 0.6425317525863647),
 ('fine', 0.6420440673828125),
 ('brilliant', 0.6286628246307373),
 ('marvelous', 0.6202216148376465),
 ('excellent', 0.6188085079193115),
 ('good', 0.6166402101516724),
 ('amazing', 0.5603896975517273)]

In [19]:
model.most_similar("awful")

[('terrible', 0.777208149433136),
 ('horrible', 0.7345371246337891),
 ('abysmal', 0.7309399247169495),
 ('atrocious', 0.7255470156669617),
 ('dreadful', 0.7129322290420532),
 ('horrendous', 0.6862552165985107),
 ('horrid', 0.6757344007492065),
 ('appalling', 0.6731106042861938),
 ('amateurish', 0.632673978805542),
 ('lousy', 0.6291776895523071)]

In [20]:
# One of the reviews we referred to often in previous write-ups was of a zombie movie,
# so let's see what words are similar/associated with the word 'zombie'
model.most_similar("zombie")

[('cannibal', 0.6610051393508911),
 ('slasher', 0.6199641227722168),
 ('horror', 0.6184082627296448),
 ('zombies', 0.613161027431488),
 ('monster', 0.6093231439590454),
 ('vampire', 0.6016378402709961),
 ('fulci', 0.5893048048019409),
 ('splatter', 0.5890129804611206),
 ('werewolf', 0.5860388278961182),
 ('chainsaw', 0.5515424013137817)]

In [21]:
len(model.wv.index2word)

16490

Now that we have a trained model with some semantic understanding of words, how should we use it?

We'll try clustering--even though according to Kaggle it doesn't offer an improvement--as a programming exercise before moving on to other methods.

Note that the Word2Vec model we trained consists of a feature vector for each word in the vocabulary.  The feature vectors can be accessed via the "syn0" object property.


In [22]:
from sklearn.cluster import KMeans

# Set "k" to be 1/5th of the vocabulary size, or an average of 5 words per cluster
wordVecs = model.wv.syn0
k = int(wordVecs.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kMeans = KMeans( n_clusters = k )
kModel = kMeans.fit_predict(wordVecs)

So now we have K clusters, and each word in the Word2Vec vocabulary has been assigned to one of the clusters.  Next we want to combine the actual words with their cluster assignments.  We can pull the words themselves from the Word2Vec object with the following property:

```python
model.wv.index2word
```

Let's ensure the list lengths match and then combine the words and their assignments into a dictionary object:

In [23]:
print(len(model.wv.index2word))
print(len(kModel))

print(model.wv.index2word[:5])
print(kModel[:5])

16490
16490
['the', 'and', 'a', 'of', 'to']
[2546  171 2750 2445 2349]


In [24]:
clusterDict = dict(zip(model.wv.index2word, kModel))

Quick visual inspection:

In [25]:
for i, k in enumerate(clusterDict.keys()):
    print(k, "=", clusterDict[k])
    
    if i > 3:
        break

the = 2546
and = 171
a = 2750
of = 2445
to = 2349


In [26]:
# Examine the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(clusterDict.values())):
        if( list(clusterDict.values())[i] == cluster ):
            words.append(list(clusterDict.keys())[i])
    print(words)


Cluster 0
['courtroom', 'pivotal', 'stealer']

Cluster 1
['francis', 'fleming', 'diaz', 'crane', 'cristina', 'regal', 'rhonda', 'reyes', 'petulia']

Cluster 2
['charged', 'cunning', 'cerebral', 'elusive', 'evolving', 'heated', 'volatile', 'comforting', 'guiding', 'shamefully', 'smarts']

Cluster 3
['underwear', 'cleavage', 'panties', 'baring', 'strut']

Cluster 4
['ancient', 'underground', 'forbidden', 'egyptian', 'occult', 'mythological', 'aztec', 'kells']

Cluster 5
['repetitive', 'tiresome', 'monotonous', 'tiring']

Cluster 6
['bang', 'stairs', 'scratch', 'climbing', 'pin', 'climb', 'ladder']

Cluster 7
['association']

Cluster 8
['madness', 'menace', 'paranoia', 'relentless', 'izo', 'increases', 'decay', 'deceit', 'treachery', 'intrigues']

Cluster 9
['creepy', 'effective', 'stylish', 'atmospheric', 'eerie', 'spooky', 'gothic', 'moody', 'claustrophobic', 'ominous', 'unnerving']


Previously when we implemented bag-of-words we counted up how many times a certain word appeared in each review.  We were hoping that word count patterns would emerge in similar reviews, and that would help us classify unseen reviews as good or bad by comparing their word count patterns.  

In this instance we are doing the same thing, but instead of counting word occurrences we are counting how many times the cluster containing a given word appears in the review.  Again, we are hoping that cluster count patterns emerge that are similar between like reviews, and that we can use this to identify unseen reviews as good or bad.  We are switching from individual words to semantically related clusters comparisons.

The first thing we need to do is write a function that returns a an array for a given review   Each entry in the array should correspond to a cluster in our set, and the values for the array entries will the number of times the cluster was found in the review text.

Next we need to collect each of the feature arrays into a single object suitable for being passed to a machine learning algorithm for training.

##### Feature array creation

In [27]:
def createFeatureArray(sentences, clusterDict):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( clusterDict.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for sentence in sentences:
        for word in sentence:
            if word in clusterDict:
                index = clusterDict[word]
                bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [28]:
trainSentences = []

for s in df.iloc[:,2]:
    trainSentences.append(createSentences(s, tokenizer, remove_stopwords = True))
    


In [29]:
k = int(wordVecs.shape[0] / 5)

# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (df.iloc[:,2].size, k), dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in trainSentences:
    train_centroids[counter,:] = createFeatureArray(review, clusterDict)
    counter += 1

In [30]:
# Init vars and params
eFolds = 10
eSeed = 10

# Use accuracy since this is a classification problem
eScore = 'accuracy'

modelName = 'RandomForestClassifier'
RandomForestClassifier(n_estimators = 100)
xTrain = train_centroids
yTrain = df.iloc[:, 1]

_DF = pd.DataFrame(columns = ['Model', 'Accuracy', 'StdDev'])
_Results = {}
_model = RandomForestClassifier(n_estimators = 100)

kFold = KFold(n_splits = eFolds, random_state = eSeed)
_Results[modelName] = cross_val_score(_model, xTrain, yTrain, cv = kFold, scoring = eScore)

_DF.loc[len(_DF)] = list(['RandomForestClassifier', _Results[modelName].mean(), _Results[modelName].std()])
display(_DF.sort_values(by = ['Accuracy', 'StdDev', 'Model'], ascending = [False, True, True]))

,Model,Accuracy,StdDev
0,RandomForestClassifier,0.8468,0.008575


# Summary

In [31]:
%%html
<style>
table {float:left}
</style>

In this write-up we accomplished the following:


1. Created a set of word embeddings from the IMDb movie review text utilizing Word2vec
2. Clustered the embeddings utilizing a K-nearest neighbors algorithm into a set of centroids
3. Trained and evaluated the models from the last write-up against the centroid feature set

And finally, here is the baseline model's performance vs. the 'centroid' model we developed in this write-up:

|Model|Accuracy|Best Params                           |
|--------|--------|-----------------------------------|
|LR      |86.35%  |{'LR__C': 0.1, 'LR__penalty': 'l1'}|
|Centroid|84.68%  |Estimators = 100                   |

Similar to the last [last write-up](./Model-06.p2.ipynb) the work in this notebook was an interesting idea to explore, but ultimately didn't result in an overall performance increase versus the baseline model.  As such this line of exploration will be rejected in favor of keeping the current base line model and accuracy rating as benchmarks moving forward.

In [ ]:
# Init vars
folds = 10
seed = 10
models = []
results = {}

# Use accuracy since this is a classification
score = 'accuracy'

# Instantiate model objects
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Create a Pandas DF to hold all our spiffy results
_df = pd.DataFrame(columns = ['Model', 'Accuracy', 'StdDev'])

# Run the models
for modelName, model in models:
    print("Training", modelName, "....")
    # Implement K-fold cross validation where K = 10
    kFold = KFold(n_splits = folds, random_state = seed)
    results[modelName] = cross_val_score(model, xTrain, yTrain, cv = kFold, scoring = score)
    _df.loc[len(_df)] = list([modelName, results[modelName].mean(), results[modelName].std()])

# Print results sorted by Mean desc, StdDev asc, Model asc
_df.sort_values(by = ['Accuracy', 'StdDev', 'Model'], ascending = [False, True, True])

Training LR ....
Training LDA ....
Training KNN ....
Training CART ....
Training NB ....
Training SVM ....
